In [ ]:
import tensorflow as tf

In [ ]:
def conv2d_block(input_tensor, n_filters, kernel_size=3):
    '''
    Adds 2 convolutional layers with the parameters passed to it

    Args:
    input_tensor (tensor) -- the input tensor
    n_filters (int) -- number of filters
    kernel_size (int) -- kernel size for the convolution

    Returns:
    tensor of output features
    '''
    x = input_tensor
    for i in range(2):
        x = tf.keras.layers.Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer='he_normal', padding='same')(x)
        x = tf.keras.layers.Activation('relu')(x)

    return x


def encoder_block(inputs, n_filters=64, pool_size=(2,2), dropout=0.3):
    '''
    Adds two convolutional blocks and then perform down sampling on output of convolutions.

    Args:
    input_tensor (tensor) -- the input tensor
    n_filters (int) -- number of filters
    kernel_size (int) -- kernel size for the convolution

    Returns:
    f - the output features of the convolution block 
    p - the maxpooled features with dropout
  '''

    f = conv2d_block(inputs, n_filters=n_filters)
    p = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(f)
    # p = tf.keras.layers.Dropout(0.3)(p)

    return f, p


def encoder(inputs):
    '''
    This function defines the encoder or downsampling path.

    Args:
    inputs (tensor) -- batch of input images

    Returns:
    p4 - the output maxpooled features of the last encoder block
    (f1, f2, f3, f4) - the output features of all the encoder blocks
    '''
    f1, p1 = encoder_block(inputs, n_filters=16,  pool_size=(2,2), dropout=0.3)
    f2, p2 = encoder_block(p1,     n_filters=32, pool_size=(2,2), dropout=0.3)
    f3, p3 = encoder_block(p2,     n_filters=64, pool_size=(2,2), dropout=0.3)
    f4, p4 = encoder_block(p3,     n_filters=128, pool_size=(2,2), dropout=0.3)

    return p4, (f1, f2, f3, f4)

def bottleneck(inputs):
    '''
    This function defines the bottleneck convolutions to extract more features before the upsampling layers.
    '''

    bottle_neck = conv2d_block(inputs, n_filters=256)

    return bottle_neck

In [ ]:
def decoder_block(inputs, conv_output, n_filters=64, kernel_size=3, strides=3, dropout=0.3):
    '''
    defines the one decoder block of the UNet

    Args:
    inputs (tensor) -- batch of input features
    conv_output (tensor) -- features from an encoder block
    n_filters (int) -- number of filters
    kernel_size (int) -- kernel size
    strides (int) -- strides for the deconvolution/upsampling
    padding (string) -- "same" or "valid", tells if shape will be preserved by zero padding

    Returns:
    c (tensor) -- output features of the decoder block
    '''
    u = tf.keras.layers.Conv2DTranspose(n_filters, kernel_size, strides=strides, padding='same')(inputs)
    c = tf.keras.layers.concatenate([u, conv_output])
    # c = tf.keras.layers.Dropout(dropout)(c)
    c = conv2d_block(c, n_filters, kernel_size=3)

    return c


def decoder(inputs, convs, output_channels):
    '''
    Defines the decoder of the UNet chaining together 4 decoder blocks. 

    Args:
    inputs (tensor) -- batch of input features
    convs (tuple) -- features from the encoder blocks
    output_channels (int) -- number of classes in the label map

    Returns:
    outputs (tensor) -- the pixel wise label map of the image
    '''

    f1, f2, f3, f4 = convs

    c6 = decoder_block(inputs, f4, n_filters=128, kernel_size=(3,3), strides=(2,2), dropout=0.3)
    c7 = decoder_block(c6,     f3, n_filters=64, kernel_size=(3,3), strides=(2,2), dropout=0.3)
    c8 = decoder_block(c7,     f2, n_filters=32, kernel_size=(3,3), strides=(2,2), dropout=0.3)
    # c9 = decoder_block(c8,     f1, n_filters=64,  kernel_size=(3,3), strides=(2,2), dropout=0.3)

    outputs = tf.keras.layers.Conv2D(output_channels, (1, 1), activation='softmax')(c8)

    return outputs

In [ ]:
OUTPUT_CHANNELS = 1

def unet():
    inputs = tf.keras.layers.Input(shape=(256, 256, 1))

    encoder_output, convs = encoder(inputs)
    bottle_neck = bottleneck(encoder_output)
    outputs = decoder(bottle_neck, convs, output_channels=OUTPUT_CHANNELS)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    return model

model = unet()

model.summary()

In [1]:
from UNet import UNet
model = UNet()
model.summary()

2021-12-06 02:01:32.020039: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-12-06 02:01:32.020117: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LT-0054): /proc/driver/nvidia/version does not exist
2021-12-06 02:01:32.021931: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 16  160         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 activation (Activation)        (None, 256, 256, 16  0           ['conv2d[0][0]']                 
                                )                                                             

In [2]:
import tensorflow as tf
def loss_fn(y_gt, y_pr):
    losses = [tf.keras.losses.MeanSquaredError()]
    loss = 0
    for loss_fn in losses:
        loss += loss_fn(y_gt, y_pr)
    return loss
    
model_temp = tf.keras.models.load_model("../sleap/model.h5", custom_objects={ 'loss_fn': loss_fn})

In [3]:
for layer in model.layers:
    try:
        print(layer.activation)
    except:
        pass

<function linear at 0x7f65b224e320>
<function relu at 0x7f65b22498c0>
<function linear at 0x7f65b224e320>
<function relu at 0x7f65b22498c0>
<function linear at 0x7f65b224e320>
<function relu at 0x7f65b22498c0>
<function linear at 0x7f65b224e320>
<function relu at 0x7f65b22498c0>
<function linear at 0x7f65b224e320>
<function relu at 0x7f65b22498c0>
<function linear at 0x7f65b224e320>
<function relu at 0x7f65b22498c0>
<function linear at 0x7f65b224e320>
<function relu at 0x7f65b22498c0>
<function linear at 0x7f65b224e320>
<function relu at 0x7f65b22498c0>
<function linear at 0x7f65b224e320>
<function relu at 0x7f65b22498c0>
<function linear at 0x7f65b224e320>
<function relu at 0x7f65b22498c0>
<function linear at 0x7f65b224e320>
<function relu at 0x7f65b22498c0>
<function linear at 0x7f65b224e320>
<function relu at 0x7f65b22498c0>
<function linear at 0x7f65b224e320>
<function relu at 0x7f65b22498c0>
<function linear at 0x7f65b224e320>
<function relu at 0x7f65b22498c0>
<function linear at 

In [4]:
for layer in model_temp.layers:
    try:
        print(layer.activation)
    except:
        pass

<function linear at 0x7f65b224e320>
<function relu at 0x7f65b22498c0>
<function linear at 0x7f65b224e320>
<function relu at 0x7f65b22498c0>
<function linear at 0x7f65b224e320>
<function relu at 0x7f65b22498c0>
<function linear at 0x7f65b224e320>
<function relu at 0x7f65b22498c0>
<function linear at 0x7f65b224e320>
<function relu at 0x7f65b22498c0>
<function linear at 0x7f65b224e320>
<function relu at 0x7f65b22498c0>
<function linear at 0x7f65b224e320>
<function relu at 0x7f65b22498c0>
<function linear at 0x7f65b224e320>
<function relu at 0x7f65b22498c0>
<function linear at 0x7f65b224e320>
<function relu at 0x7f65b22498c0>
<function linear at 0x7f65b224e320>
<function relu at 0x7f65b22498c0>
<function linear at 0x7f65b224e320>
<function relu at 0x7f65b22498c0>
<function linear at 0x7f65b224e320>
<function relu at 0x7f65b22498c0>
<function linear at 0x7f65b224e320>
<function relu at 0x7f65b22498c0>
<function linear at 0x7f65b224e320>
<function relu at 0x7f65b22498c0>
<function linear at 

In [8]:
model_temp.loss

<function __main__.loss_fn(y_gt, y_pr)>